<a href="https://colab.research.google.com/github/edulapalle/CS584-ML-HW3/blob/main/ML_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Santosh Reddy Edulapalle**

ML-HW3

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Loading dataset

In [3]:
#required libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

path = "/content/drive/MyDrive/A20501739_CollectedData_annotation.csv"
df = pd.read_csv(path)
#head
df.head(5)

,scorer,Unnamed: 1,Unnamed: 2,annotation,annotation.1,annotation.2,annotation.3,annotation.4,annotation.5,annotation.6,...,annotation.26,annotation.27,annotation.28,annotation.29,annotation.30,annotation.31,annotation.32,annotation.33,annotation.34,annotation.35
0,individuals,NaN,NaN,mouse1,mouse1,mouse1,mouse1,mouse1,mouse1,mouse1,...,mouse2,mouse2,mouse2,mouse2,mouse2,mouse2,mouse2,mouse2,mouse2,mouse2
1,bodyparts,NaN,NaN,topleft,topleft,rightdown,rightdown,nose,nose,leftear,...,rightear,rightear,leftHip,leftHip,rightHip,rightHip,tailBase,tailBase,tailEnd,tailEnd
2,coords,NaN,NaN,x,y,x,y,x,y,x,...,x,y,x,y,x,y,x,y,x,y
3,labeled-data,23.0,A_male_in_a_new_cage_face_view_3_2022-08-10_15...,162.22631573784358,187.5751745878016,319.8845247452017,320.6233217013282,194.86484184217267,196.65162367954176,186.83637763167508,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,labeled-data,23.0,A_male_in_a_new_cage_face_view_3_2022-08-10_15...,268.0451131495447,177.5942613599138,347.3237403991978,240.86470426107923,279.4795305413216,209.61063005688908,287.86476996195796,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Handling Null values**

Since our objective is to find the center of the image by taking average of topleft and rightbottom corners, we will simply take all NA values to be 0 so we don't get errors while doing calculations with any null values.

In [4]:
#replacing all NA with 0
df.fillna(0,inplace = True)
#df['annotation.26'].isnull().sum()
df.head(5)

,scorer,Unnamed: 1,Unnamed: 2,annotation,annotation.1,annotation.2,annotation.3,annotation.4,annotation.5,annotation.6,...,annotation.26,annotation.27,annotation.28,annotation.29,annotation.30,annotation.31,annotation.32,annotation.33,annotation.34,annotation.35
0,individuals,0.0,0,mouse1,mouse1,mouse1,mouse1,mouse1,mouse1,mouse1,...,mouse2,mouse2,mouse2,mouse2,mouse2,mouse2,mouse2,mouse2,mouse2,mouse2
1,bodyparts,0.0,0,topleft,topleft,rightdown,rightdown,nose,nose,leftear,...,rightear,rightear,leftHip,leftHip,rightHip,rightHip,tailBase,tailBase,tailEnd,tailEnd
2,coords,0.0,0,x,y,x,y,x,y,x,...,x,y,x,y,x,y,x,y,x,y
3,labeled-data,23.0,A_male_in_a_new_cage_face_view_3_2022-08-10_15...,162.22631573784358,187.5751745878016,319.8845247452017,320.6233217013282,194.86484184217267,196.65162367954176,186.83637763167508,...,0,0,0,0,0,0,0,0,0,0
4,labeled-data,23.0,A_male_in_a_new_cage_face_view_3_2022-08-10_15...,268.0451131495447,177.5942613599138,347.3237403991978,240.86470426107923,279.4795305413216,209.61063005688908,287.86476996195796,...,0,0,0,0,0,0,0,0,0,0


In [5]:
cols = ["scorer","id","img","m1_topleft_x","m1_topleft_y","m1_rightdown_x","m1_rightdown_y","m1_nose_x","m1_nose_y","m1_leftear_x","m1_leftear_y","m1_rightear_x","m1_rightear_y","m1_leftHip_x","m1_leftHip_y","m1_rightHip_x","m1_rightHip_y","m1_tailBase_x","m1_tailBase_y","m1_tailEnd_x","m1_tailEnd_y","m2_topleft_x","m2_topleft_y","m2_rightdown_x","m2_rightdown_y","m2_nose_x","m2_nose_y","m2_leftear_x","m2_leftear_y","m2_rightear_x","m2_rightear_y","m2_leftHip_x","m2_leftHip_y","m2_rightHip_x","m2_rightHip_y","m2_tailBase_x","m2_tailBase_y","m2_tailEnd_x","m2_tailEnd_y"]

In [6]:
#renaming the columns
#df.columns = cols
df.set_axis(cols, axis='columns', inplace=True)
df.head(5)

,scorer,id,img,m1_topleft_x,m1_topleft_y,m1_rightdown_x,m1_rightdown_y,m1_nose_x,m1_nose_y,m1_leftear_x,...,m2_rightear_x,m2_rightear_y,m2_leftHip_x,m2_leftHip_y,m2_rightHip_x,m2_rightHip_y,m2_tailBase_x,m2_tailBase_y,m2_tailEnd_x,m2_tailEnd_y
0,individuals,0.0,0,mouse1,mouse1,mouse1,mouse1,mouse1,mouse1,mouse1,...,mouse2,mouse2,mouse2,mouse2,mouse2,mouse2,mouse2,mouse2,mouse2,mouse2
1,bodyparts,0.0,0,topleft,topleft,rightdown,rightdown,nose,nose,leftear,...,rightear,rightear,leftHip,leftHip,rightHip,rightHip,tailBase,tailBase,tailEnd,tailEnd
2,coords,0.0,0,x,y,x,y,x,y,x,...,x,y,x,y,x,y,x,y,x,y
3,labeled-data,23.0,A_male_in_a_new_cage_face_view_3_2022-08-10_15...,162.22631573784358,187.5751745878016,319.8845247452017,320.6233217013282,194.86484184217267,196.65162367954176,186.83637763167508,...,0,0,0,0,0,0,0,0,0,0
4,labeled-data,23.0,A_male_in_a_new_cage_face_view_3_2022-08-10_15...,268.0451131495447,177.5942613599138,347.3237403991978,240.86470426107923,279.4795305413216,209.61063005688908,287.86476996195796,...,0,0,0,0,0,0,0,0,0,0


We will try to copy data to dummy df and perfrom some operations.


In [11]:
#top-3 rows are just additional data, we don't require them.
df_copy = df
df_copy = df_copy[3:]


df_copy.head(5)

,scorer,id,img,m1_topleft_x,m1_topleft_y,m1_rightdown_x,m1_rightdown_y,m1_nose_x,m1_nose_y,m1_leftear_x,...,m2_rightear_x,m2_rightear_y,m2_leftHip_x,m2_leftHip_y,m2_rightHip_x,m2_rightHip_y,m2_tailBase_x,m2_tailBase_y,m2_tailEnd_x,m2_tailEnd_y
3,labeled-data,23.0,A_male_in_a_new_cage_face_view_3_2022-08-10_15...,162.22631573784358,187.5751745878016,319.8845247452017,320.6233217013282,194.86484184217267,196.65162367954176,186.83637763167508,...,0,0,0,0,0,0,0,0,0,0
4,labeled-data,23.0,A_male_in_a_new_cage_face_view_3_2022-08-10_15...,268.0451131495447,177.5942613599138,347.3237403991978,240.86470426107923,279.4795305413216,209.61063005688908,287.86476996195796,...,0,0,0,0,0,0,0,0,0,0
5,labeled-data,23.0,A_male_in_a_new_cage_face_view_3_2022-08-10_15...,146.84028879670976,189.79097324447577,338.9385009785614,285.0777848426165,156.75011720291639,220.28275295588077,181.90583546482554,...,0,0,0,0,0,0,0,0,0,0
6,labeled-data,23.0,A_male_in_a_new_cage_face_view_3_2022-08-10_15...,146.0779943039246,202.74997962182283,366.38110271882596,348.34822774378193,159.79929517405688,269.0696004941288,174.28289053697426,...,0,0,0,0,0,0,0,0,0,0
7,labeled-data,23.0,A_male_in_a_new_cage_face_view_3_2022-08-10_15...,232.2172719886438,174.54508338877326,393.06140996630535,230.95487585487263,240.60251140928023,214.18439701359978,242.12710039485046,...,0,0,0,0,0,0,0,0,0,0


In [19]:
#adding new column
df_copy["m1_x"]= (df_copy["m1_topleft_x"].astype(float)+df_copy["m1_rightdown_x"].astype(float))/2
df_copy["m1_y"]= (df_copy["m1_topleft_y"].astype(float)+df_copy["m1_rightdown_y"].astype(float))/2
df_copy["m2_x"]= (df_copy["m2_topleft_x"].astype(float)+df_copy["m2_rightdown_x"].astype(float))/2
df_copy["m2_y"]= (df_copy["m2_topleft_y"].astype(float)+df_copy["m2_rightdown_y"].astype(float))/2

df_copy.head(5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

,scorer,id,img,m1_topleft_x,m1_topleft_y,m1_rightdown_x,m1_rightdown_y,m1_nose_x,m1_nose_y,m1_leftear_x,...,m2_rightHip_x,m2_rightHip_y,m2_tailBase_x,m2_tailBase_y,m2_tailEnd_x,m2_tailEnd_y,m1_x,m1_y,m2_x,m2_y
3,labeled-data,23.0,A_male_in_a_new_cage_face_view_3_2022-08-10_15...,162.22631573784358,187.5751745878016,319.8845247452017,320.6233217013282,194.86484184217267,196.65162367954176,186.83637763167508,...,0,0,0,0,0,0,241.055420,254.099248,0.0,0.0
4,labeled-data,23.0,A_male_in_a_new_cage_face_view_3_2022-08-10_15...,268.0451131495447,177.5942613599138,347.3237403991978,240.86470426107923,279.4795305413216,209.61063005688908,287.86476996195796,...,0,0,0,0,0,0,307.684427,209.229483,0.0,0.0
5,labeled-data,23.0,A_male_in_a_new_cage_face_view_3_2022-08-10_15...,146.84028879670976,189.79097324447577,338.9385009785614,285.0777848426165,156.75011720291639,220.28275295588077,181.90583546482554,...,0,0,0,0,0,0,242.889395,237.434379,0.0,0.0
6,labeled-data,23.0,A_male_in_a_new_cage_face_view_3_2022-08-10_15...,146.0779943039246,202.74997962182283,366.38110271882596,348.34822774378193,159.79929517405688,269.0696004941288,174.28289053697426,...,0,0,0,0,0,0,256.229549,275.549104,0.0,0.0
7,labeled-data,23.0,A_male_in_a_new_cage_face_view_3_2022-08-10_15...,232.2172719886438,174.54508338877326,393.06140996630535,230.95487585487263,240.60251140928023,214.18439701359978,242.12710039485046,...,0,0,0,0,0,0,312.639341,202.749980,0.0,0.0


Train-Test-split

Splitting the dataset to 70-30 for training and evaluation.

In [7]:
#top-3 rows are just additional data, we don't require them.
df1 = df[3:]
df1.head()

,scorer,id,img,m1_topleft_x,m1_topleft_y,m1_rightdown_x,m1_rightdown_y,m1_nose_x,m1_nose_y,m1_leftear_x,...,m2_rightear_x,m2_rightear_y,m2_leftHip_x,m2_leftHip_y,m2_rightHip_x,m2_rightHip_y,m2_tailBase_x,m2_tailBase_y,m2_tailEnd_x,m2_tailEnd_y
3,labeled-data,23.0,A_male_in_a_new_cage_face_view_3_2022-08-10_15...,162.22631573784358,187.5751745878016,319.8845247452017,320.6233217013282,194.86484184217267,196.65162367954176,186.83637763167508,...,0,0,0,0,0,0,0,0,0,0
4,labeled-data,23.0,A_male_in_a_new_cage_face_view_3_2022-08-10_15...,268.0451131495447,177.5942613599138,347.3237403991978,240.86470426107923,279.4795305413216,209.61063005688908,287.86476996195796,...,0,0,0,0,0,0,0,0,0,0
5,labeled-data,23.0,A_male_in_a_new_cage_face_view_3_2022-08-10_15...,146.84028879670976,189.79097324447577,338.9385009785614,285.0777848426165,156.75011720291639,220.28275295588077,181.90583546482554,...,0,0,0,0,0,0,0,0,0,0
6,labeled-data,23.0,A_male_in_a_new_cage_face_view_3_2022-08-10_15...,146.0779943039246,202.74997962182283,366.38110271882596,348.34822774378193,159.79929517405688,269.0696004941288,174.28289053697426,...,0,0,0,0,0,0,0,0,0,0
7,labeled-data,23.0,A_male_in_a_new_cage_face_view_3_2022-08-10_15...,232.2172719886438,174.54508338877326,393.06140996630535,230.95487585487263,240.60251140928023,214.18439701359978,242.12710039485046,...,0,0,0,0,0,0,0,0,0,0


In [13]:
#splitting dataset
train_df = df.sample(frac=0.7, random_state=0)
test_df = df.drop(train_df.index)

In [17]:
print("training df size: "+str(train_df.shape))
print("testing df size: "+str(test_df.shape))


training df size: (90, 39)
testing df size: (38, 39)
